In [45]:
#Packages being downloaded
import csv
import pandas as pd
import re
import xml.etree.ElementTree as ET

from tkinter import filedialog
from tkinter import *

In [2]:
#%%python
def getFile():
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("text files","*.csv"),("all files","*.*")))
    root.destroy()
    return(root.filename)

In [3]:
def readCSV(csvName):
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

In [16]:
def getCat(df,colorName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) <5 and colorName=="": 
                colorName=df.columns[x]
        x= x+1
    return colorName

In [25]:
def getCatCol(df,colorName):
    return df[colorName].astype('category').cat.categories.tolist()

In [28]:
def typeDefiner(df):
    colList = df.columns
    colList
    x = 0
    colNames = []
    for name in colList:
        if df.dtypes[x] == 'object':# or df.dtypes[x] == 'int64':
            pass
        else:
            colNames.append(colList[x])
                
        x = x + 1
    print("objects removed")
    print(colNames)
    return colNames

In [ ]:
def toNumeric(sdtFile):

    import xml.etree.ElementTree as ET
    #CLEARS COLOR OUT NOW WE CAN INSERT ANYTHING
    file = ET.parse(sdtFile)
    root = file.getroot()
    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        elem.clear()

    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)
   #Try to write here
    for elem in root.findall('.//Color/RGB'):
        check_req_elems = elem.find('.//Function[@type="Text Field To Value"]')
        check_req_elems.type = "Linear Interpolation"
        check_req_elems.set("type","Linear Interpolation")
        attrib = {'type': 'BoundField'}
        minmax = ET.SubElement(check_req_elems, 'MinMax', attrib)
        minmax.text= "\n\t\t\t\t\t\t\t"
        min = ET.SubElement(minmax, 'Min')
        min.text = '0'
        min.tail="\n\t\t\t\t\t\t"
        minmax.tail="\n\t\t\t\t\t"
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)
    print("color fixed")

In [59]:
def XMLParser(colNames, sdtFile,colorName):
    print(colNames)

    tree = ET.parse(sdtFile)
    root = tree.getroot()
    count=0;
    print(len(colNames))
    for elem in root.iter('InputField'):
        print(count)
        #count+=1;
        #check_req_elem = elem.find('.//InputField')
        #if check_req_elem.get('name')!="color":
        elem.set('field', colNames[count])
        #else:
         #   elem.set('field', colorName)
        count+=1;
    #tree.write('New DAL 900.sdt')
    tree.write('emptySDT.sdt')
    print("XML updated")

In [69]:
def runAll(sdtFile):
    colorCol= None
    colorName=""
    csv = getFile()
    df = readCSV(csv)
    colorName=getCat(df,colorName)
    colorCol=getCatCol(df,colorName)
    colNames = typeDefiner(df)
    colNames.append(colorName)
    XMLParser(colNames, sdtFile,colorName)
    setupColor(colorCol, sdtFile)
   # csv = getFile()
#readCSV(csv)

In [70]:
#%%python
runAll('emptySDT.sdt')

reading csv done
objects removed
['Height', 'Age', 'GPA']
['Height', 'Age', 'GPA', 'Gender']
4
0
1
2
3
XML updated
F
M


### Extra functions

In [4]:
def elevationFinder(df):
    if df.columns.contains("elevation") or df.columns.contains("Elevation") or df.columns.contains("height"):
        longitude = df["Longitude"]
    else:
        longitude = "none found"
    return longitude

In [3]:
def latFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LAT', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            lat = df.columns[x]
            return lat
        else:
            x = x + 1
            pass

In [2]:
def longFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LONG', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            long = df.columns[x]
            return long
        else:
            x = x + 1
            pass

In [67]:
def setupColor(colorCol,sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    x=0
    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        for element in elem.findall('KeyValuePair/Key'):
            element.text=colorCol[x]
            print(colorCol[x])
            x= x+1
            
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)


NameError: name 'colorCol' is not defined